# Scenario 1 Data Creation
This notebook creates 4 datasets to be used for Scenario 1 and then solves each problem to ensure that the results make reasonable sense.

In [295]:
import pandas as pd
import names
import random
import requests
import re
from numpy.random import choice
from datetime import datetime, timedelta
import hashlib

In [518]:
bureaus_weighted = [('Administration (A)',3,2,5),
('African Affairs (AF)',10,4,2),
('Allowances (A/OPR/ALS)',1,2,1),
('Arms Control, Verification and Compliance (AVC)',2,2,1),
('Authentications',1,5,1),
('Budget and Planning (BP)',2,4,2),
('Chief Information Officer (CIO)',1,7,1),
('Chief Economist, of the Department',1,8,1),
('Chief of Staff',1,8,1),
('Civil Rights, Office of',1,3,2),
('Comptroller and Global Financial Services (CGFS)',5,2,5),
('Conflict and Stabilization Operations (CSO)',5,2,4),
('Consular Affairs (CA)',10,1,8),
('Counterterrorism (CT)',2,3,6),
('Democracy, Human Rights, and Labor (DRL)',10,2,3),
('Department Spokesperson',1,9,1),
('Diplomatic Reception Rooms',1,5,1),
('Diplomatic Security (DS)',10,2,10),
('Director General of the Foreign Service and Director of Human Resources (DGHR)',1,7,2),
('East Asian and Pacific Affairs (EAP)',10,5,8),
('Economic and Business Affairs (EB)',5,5,3),
('Educational and Cultural Affairs (ECA)',5,3,6),
('Energy Resources (ENR)',1,7,2),
('European and Eurasian Affairs (EUR)',10,4,10),
('Executive Secretariat (S/ES)',2,4,3),
('Foreign Assistance (F)',2,5,8),
('Foreign Missions (OFM)',1,5,3),
('Foreign Service Institute (FSI)',5,4,4),
('Global AIDS Coordinator (S/GAC)',1,7,2),
('Global Criminal Justice (GCJ)',1,7,2),
('Global Engagement Center(GEC)',1,7,4),
('Global Food Security',1,7,2),
("Global Women's Issues (S/GWI)",1,7,2),
('Global Youth Issues (GYI)',1,7,2),
('Human Resources (DGHR)',10,1,6),
('Information Resource Management (IRM)',10,1,10),
('Inspector General (OIG)',1,5,2),
('Intelligence and Research (INR)',5,4,3),
('International Information Programs (IIP)',5,3,5),
('International Narcotics and Law Enforcement Affairs (INL)',5,3,7),
('International Organization Affairs (IO)',5,5,3),
('International Security and Nonproliferation (ISN)',5,3,2),
('Legal Adviser (L)',1,6,1),
('Legislative Affairs (H)',1,6,2),
('Management Policy, Rightsizing and Innovation (PRI)',2,3,3),
('Medical Services (MED)',2,3,5),
('Mission to the United Nations (USUN)',2,6,5),
('Near Eastern Affairs (NEA)',10,5,7),
('Oceans and International Environmental and Scientific Affairs(OES)',1,5,2),
('Office of Terrorism Finance and Economic Sanctions Policy',1,6,1),
('Ombudsman, Office of',1,5,1),
('Overseas Buildings Operations (OBO)',5,5,10),
('Policy, Planning, and Resources for Public Diplomacy and Public Affairs (PPR)',2,4,4),
('Policy Planning Staff (S/P)',1,7,3),
('Political-Military Affairs (PM)',5,3,8),
('Population, Refugees, and Migration (PRM)',1,6,2),
('Protocol (S/CPR)',1,5,5),
('Public Affairs (PA)',5,5,7),
('Quadrennial Diplomacy and Development Review (QDDR)',2,6,7),
('Science & Technology Adviser (STAS)',1,6,1),
('South and Central Asian Affairs (SCA)',10,6,6),
('Trafficking in Persons (TIP)',1,5,2),
('Western Hemisphere Affairs (WHA)',10,5,7)]

In [519]:
bureaus_weighted = [i for i in bureaus_weighted if '(' in i[0]]

In [520]:
def size_adjust(x):
    if x >= 5:
        x = x * 3
    return x

In [521]:
b_df = pd.DataFrame(bureaus_weighted, columns=['Bureau','Size','Seniority','Budget'])
# b_df['Size3x'] = b_df.Size.apply(lambda x: size_adjust(x))
b_df['Size_cum'] = b_df['Size'].cumsum()

In [522]:
min_bureau = 0
max_bureau = b_df.Size_cum.max()

# Generate Staffing by Bureau

In [523]:
def gen_person():
    # random variables
    random_cat = random.random()
    random_bureau = random.randint(min_bureau,max_bureau)
    random_senior = random.random()
    random_ws_trail = random.random()
    random_ws_start = random.random()    
    random_last_only = random.random()
    random_grade_nodash = random.random()
    random_grade_space = random.random()
    random_title_adjust = random.random()
    random_TDY = random.random()

    # select Bureau
    Bureau = b_df.loc[b_df['Size_cum'] >= random_bureau][:1].Bureau.values[0]    
    
    seniority_weighting = b_df.loc[b_df['Bureau'] == Bureau].Seniority.values[0]
    random_mgmt = random.randint(0,10)
    if random_mgmt >= seniority_weighting:
        Title = 'Team Member'
    else:
        Title = 'Manager'        
    
    # select Gender
    Gender = random.choice(['male','female'])
    
    # select Name
    if random_last_only > 0.9:
        Name = names.get_last_name()
    else:
        Name = names.get_full_name(gender=Gender)
        
    # add random trailing space to Name
    if random_ws_trail > 0.7:
        num_spaces = random.randint(1,5)
        for i in range(num_spaces):
            Name += ' '
    
    # add random leading space to Name
    if random_ws_start > 0.7:
        num_spaces = random.randint(1,5)
        for i in range(num_spaces):
            Name = ' ' + Name
    
    # generate years in service
    YearsService = random.randint(1,25)
    
    # generate category/grade
    if random_cat < 0.2:
        Category = 'Contractor'
        Grade = 'N/A'
    elif random_cat < 0.5:
        Category = 'Civil Service'
        if random_senior > 0.95:
            gs = random.randint(1,5)
            Grade = 'SES-' + str(gs)
            YearsService = random.randint(18,25)
            Title = 'Executive'
        else:
            if Title == 'Manager':
                gs = random.randint(13,15)
            else:
                gs = random.randint(7,13)
            Grade = 'GS-' + str(gs)
    else:
        Category = 'Foreign Service'
        if random_senior > 0.95:
            fs = random.randint(1,5)
            Grade = 'SFS-' + str(fs)
            YearsService = random.randint(18,25)
            Title = 'Executive'
        else:
            if Title == 'Manager':
                fs = random.randint(1,3)
            else:
                fs = random.randint(4,9)
            Grade = 'FS-' + str(fs)
    
    # Remove dash from Grade
    if Grade != 'N/A':
        if random_grade_nodash > 0.9:
            Grade = Grade.replace('-','')
        elif random_grade_space > 0.9:
            Grade = Grade.replace('-',' ')
    
    # Edit title
    if random_title_adjust > 0.9:
        if Title == 'Manager':
            Title = 'Mgmt'
        elif Title == 'Team Member':
            Title = 'Analyst'
            
    if random_TDY > 0.95:
        TDY = 'Yes'
    else:
        TDY = 'No'
    
    EE = {'Gender': Gender,
          'Name': Name,
          'Bureau': Bureau,
          'YearsService': YearsService,
          'Category': Category,
          'Grade': Grade,
          'Title': Title,
          'TDY': TDY}

    return EE

In [524]:
cont = False
while cont == False:
    check = gen_person()
    if check['Category'] != 'Contractor':
        if ' ' in check['Grade']:
            cont = True

In [525]:
EEs = []
for i in range(10000):
    EEs.append(gen_person())

In [526]:
EE = pd.DataFrame(columns=['Name','Gender','Grade','Bureau','Category','Title','YearsService','TDY'])

In [527]:
EE = EE.from_records(EEs)

In [528]:
EE = EE[['Bureau','Gender','Grade','Title','Name','YearsService','TDY']]

In [529]:
EE.to_csv('../Class Files/Scenario 1/Staffing_by_Bureau.csv', index=False)

In [530]:
EE.groupby('Bureau').Name.count().sort_values(ascending=False)

Bureau
European and Eurasian Affairs (EUR)                                               489
Near Eastern Affairs (NEA)                                                        479
Democracy, Human Rights, and Labor (DRL)                                          476
Western Hemisphere Affairs (WHA)                                                  468
Diplomatic Security (DS)                                                          464
Information Resource Management (IRM)                                             460
Human Resources (DGHR)                                                            451
East Asian and Pacific Affairs (EAP)                                              450
South and Central Asian Affairs (SCA)                                             445
Consular Affairs (CA)                                                             442
African Affairs (AF)                                                              419
International Narcotics and Law Enforcement Aff

# Generate Budget by Bureau

In [531]:
base = 100000

In [532]:
base*(10**3)

100000000

In [533]:
def choose_budget(x):
    base = 100000
    if x < 2:
        b = random.randint(base,base*(10**1))
    elif x < 5:
        b = random.randint(base*(10**1),base*(10**2))
    elif x < 7:
        b = random.randint(base*(10**2),base*(10**3))
    else:
        b = random.randint(base*(10**3),base*(10**4))        
    return b

def get_acronym(x):
    return x.split('(')[1].split(')')[0]

In [534]:
b_df['Budget Size (USD)'] = b_df.Budget.apply(lambda x: choose_budget(x))
b_df['Bureau Acronym'] = b_df.Bureau.apply(lambda x: get_acronym(x))

In [535]:
budget = b_df[['Bureau Acronym','Budget Size (USD)']]

In [536]:
budget.to_csv('../Class Files/Scenario 1/Budget_by_Bureau.csv', index=False)

# Generate Real Estate Allocations

In [537]:
annexes = []
for i in range(1,30):
    annexes.append('SA{}'.format(i))

In [538]:
20*20

400

In [539]:
def gen_office(Bureau):
    # random variables
    random_building = random.random()
    
    if random_building < 0.65:
        Building = 'HST'
        Location = random.randint(1000,7999)
    else:
        Building = 'SA{}'.format(random.randint(1,10))
        Location = random.randint(100,5000)
    
    Size = round(random.randint(500,2000),-1)

    office = {'Bureau': Bureau,
          'Building': Building,
          'Size': Size,
          'Location': Location}

    return office

In [540]:
offices = []
for i in bureaus_weighted:
    Bureau = i[0]
    for off in range(1,i[1]*3):
        offices.append(gen_office(Bureau))

In [541]:
RE = pd.DataFrame(columns=['Building','Bureau','Location','Size'])
RE = df.from_records(offices)

In [542]:
RE.groupby(['Bureau']).Size.sum()

Bureau
Administration (A)                                                                 9840
African Affairs (AF)                                                              35500
Allowances (A/OPR/ALS)                                                             2160
Arms Control, Verification and Compliance (AVC)                                    6220
Budget and Planning (BP)                                                           7490
Chief Information Officer (CIO)                                                    3260
Comptroller and Global Financial Services (CGFS)                                  14990
Conflict and Stabilization Operations (CSO)                                       16010
Consular Affairs (CA)                                                             34980
Counterterrorism (CT)                                                              7010
Democracy, Human Rights, and Labor (DRL)                                          36090
Diplomatic Security (DS) 

In [543]:
RE.to_csv('../Class Files/Scenario 1/DoS_Real_Estate_Allocation.csv', index=False)

# Generate Papers by Bureau

In [582]:
def gen_paper():
    # random variables
    random_bureau = random.randint(min_bureau,max_bureau)   

    # select Bureau
    Bureau = b_df.loc[b_df['Size_cum'] >= random_bureau][:1].Bureau.values[0]  
    
    startdate = datetime(year=2018,month=1,day=1)
    Date = startdate+timedelta(random.randint(1,364))
    Date = Date.strftime('%m-%d-%Y')
    
    hash_seed = random.random()
    Paper_ID = hashlib.md5(str(hash_seed).encode('utf-8')).hexdigest()

    paper = {'Bureau': Bureau,
             'Date': Date,
             'Paper ID': Paper_ID}

    return paper

In [593]:
papers = []
for i in range(15000):
    papers.append(gen_paper())

In [594]:
paper = pd.DataFrame(columns=['Bureau','Date','Paper ID'])
paper = df.from_records(papers)

In [595]:
paper.groupby('Bureau')['Paper ID'].count().sort_values(ascending=False)

Bureau
East Asian and Pacific Affairs (EAP)                                              713
Information Resource Management (IRM)                                             699
Near Eastern Affairs (NEA)                                                        697
European and Eurasian Affairs (EUR)                                               694
Consular Affairs (CA)                                                             685
Western Hemisphere Affairs (WHA)                                                  679
Diplomatic Security (DS)                                                          670
Human Resources (DGHR)                                                            668
African Affairs (AF)                                                              668
South and Central Asian Affairs (SCA)                                             647
Democracy, Human Rights, and Labor (DRL)                                          640
Comptroller and Global Financial Services (CGFS

In [596]:
paper.to_csv('../Class Files/Scenario 1/SES_Papers_Processed_2018.csv', index=False)

# Checking
Now we should check results using the four dataframes:

* `EE`
* `budget`
* `RE`
* `paper`

### Problem 1
Which bureaus, with respect to overall operating budget (the portion of their budget which excludes human resource cost), are least well-staffed?

In [555]:
EE['Bureau Acronym'] = EE.Bureau.apply(lambda x: get_acronym(x))

In [556]:
num_EEs = EE.groupby('Bureau Acronym').Bureau.count()
num_EEs = pd.DataFrame(num_EEs)

In [557]:
budget = budget.join(num_EEs, on='Bureau Acronym')
budget['budget_per_EE'] = budget['Budget Size (USD)'] / budget['Bureau']

In [558]:
budget.sort_values('budget_per_EE', ascending=True)[:10]

,Bureau Acronym,Budget Size (USD),Bureau,budget_per_EE
3,AVC,549436,89,6173.438202
10,DRL,3061496,476,6431.714286
1,AF,2883929,419,6882.885442
2,A/OPR/ALS,407713,46,8863.326087
35,L,470522,51,9225.921569
33,IO,2437063,245,9947.195918
34,ISN,2595371,212,12242.316038
12,DGHR,7483067,493,15178.634888
21,FSI,3852414,247,15596.817814
14,EB,4312936,240,17970.566667


### Problem 2
Which bureaus have the highest proportion of managerial staff relative to their number of action officers?

In [563]:
titles = pd.DataFrame(EE.groupby(['Bureau','Title']).Name.count())
titles = titles.reset_index()

In [564]:
titles = titles.replace('Mgmt','Manager')
titles = titles.replace('Executive','Manager')
titles = titles.replace('Analyst','Team Member')

In [565]:
titles = titles.groupby(['Title','Bureau']).Name.sum().unstack().transpose().reset_index()
titles['ratio'] = titles['Team Member'] / titles['Manager']
titles.sort_values('ratio',ascending=True)[:10]

Title,Bureau,Manager,Team Member,ratio
23,Global Criminal Justice (GCJ),31,9,0.290323
26,Global Youth Issues (GYI),35,14,0.400000
39,Mission to the United Nations (USUN),72,29,0.402778
5,Chief Information Officer (CIO),33,15,0.454545
36,Legislative Affairs (H),28,15,0.535714
49,Quadrennial Diplomacy and Development Review (...,49,27,0.551020
22,Global AIDS Coordinator (S/GAC),38,22,0.578947
25,Global Women's Issues (S/GWI),43,25,0.581395
16,Energy Resources (ENR),29,17,0.586207
50,Science & Technology Adviser (STAS),30,18,0.600000


### Problem 3
Which bureaus receive the highest workload with respect to their overall resourcing?

In [570]:
titles_b = titles.set_index('Bureau')

In [597]:
paper_ = pd.DataFrame(paper.groupby('Bureau')['Paper ID'].count()).reset_index()
paper_ = paper_.join(titles_b, on='Bureau')

In [598]:
paper_['Count Papers'] = paper_['Paper ID'] 
paper_['ratio'] = paper_['Count Papers'] / paper_['Team Member']

In [599]:
paper_.sort_values('ratio', ascending=False)[:10]

,Bureau,Paper ID,Manager,Team Member,ratio,Count Papers
23,Global Criminal Justice (GCJ),63,31,9,7.000000,63
36,Legislative Affairs (H),89,28,15,5.933333,89
49,Quadrennial Diplomacy and Development Review (...,155,49,27,5.740741,155
26,Global Youth Issues (GYI),78,35,14,5.571429,78
39,Mission to the United Nations (USUN),138,72,29,4.758621,138
5,Chief Information Officer (CIO),66,33,15,4.400000,66
16,Energy Resources (ENR),69,29,17,4.058824,69
41,Oceans and International Environmental and Sci...,73,16,18,4.055556,73
46,"Population, Refugees, and Migration (PRM)",73,17,19,3.842105,73
29,Inspector General (OIG),76,14,20,3.800000,76


### Problem 4
Which bureaus are most often contributing staff to task forces or forced to send staff on TDY?

In [600]:
tdy = pd.DataFrame(EE.groupby(['Bureau','TDY']).Name.count()).reset_index()
tdy = tdy[tdy['TDY'] == 'Yes']

In [602]:
tdy.sort_values('Name', ascending=False)[:10]

,Bureau,TDY,Name
23,Diplomatic Security (DS),Yes,32
105,Western Hemisphere Affairs (WHA),Yes,28
55,Human Resources (DGHR),Yes,27
57,Information Resource Management (IRM),Yes,26
101,South and Central Asian Affairs (SCA),Yes,25
81,Near Eastern Affairs (NEA),Yes,24
27,East Asian and Pacific Affairs (EAP),Yes,24
21,"Democracy, Human Rights, and Labor (DRL)",Yes,22
3,African Affairs (AF),Yes,20
35,European and Eurasian Affairs (EUR),Yes,17


### Problem 5
Which bureaus have excess office space that might be allocated to other, less well-equipped bureaus?

In [614]:
RE_size = pd.DataFrame(RE.groupby('Bureau').Size.sum()).reset_index()

In [617]:
titles['Total EEs'] = titles['Manager'] + titles['Team Member']
titles_ = titles[['Bureau','Total EEs']]
RE_size = RE_size.join(titles_.set_index('Bureau'), on='Bureau')

In [619]:
RE_size['Ratio'] = RE_size.Size / RE_size['Total EEs']

In [624]:
RE_size.sort_values('Ratio',ascending=False)[:10]

,Bureau,Size,Total EEs,Ratio
46,"Population, Refugees, and Migration (PRM)",3450,36,95.833333
19,Foreign Assistance (F),7750,86,90.116279
27,Human Resources (DGHR),40280,451,89.312639
31,International Information Programs (IIP),19820,227,87.312775
14,Economic and Business Affairs (EB),20400,240,85.000000
1,African Affairs (AF),35500,419,84.725537
48,Public Affairs (PA),19020,225,84.533333
4,Budget and Planning (BP),7490,90,83.222222
53,Western Hemisphere Affairs (WHA),38840,468,82.991453
21,Foreign Service Institute (FSI),20490,247,82.955466
